### PART 1

In [ ]:
# reading input
f = open('inputs/9.txt')
raw_file = f.read().strip()

id = 0
processed_file = []

# for alternating between file and free space
isFile = True

for i in raw_file:
    i = int(i)
    if isFile:
        # adding i instances of id to the processed file
        processed_file += [id] * i
        id += 1
        isFile = False
    else: 
        # adding i blocks of None to the processed file
        # at the end, None will be empty space and non-none values are files
        processed_file += [None] * i
        isFile = True

print(processed_file)

# utility function to swap two locations in processed file
def swap(i1, i2):
    processed_file[i1], processed_file[i2] = processed_file[i2], processed_file[i1]

# to run a two-pointer loop from start and end of file
end = len(processed_file) - 1
start = 0

# find the first instance of free space
while processed_file[start] != None:
    start += 1

# find the first instance of file from the end
while processed_file[end] == None:
    end -= 1

# standard two-pointer loop
while start <= end:
    # we swap the current file with the free space
    swap(start, end)
    # increment the start pointer to the next free space
    while processed_file[start] != None:
       start += 1

    # decrement the end pointer to the next file from the end
    while processed_file[end] == None:
        end -= 1

checksum = 0
for index, s in enumerate(processed_file):
    if s != None:
        checksum += index * s

print(checksum)

[0, 0, 0, 0, 0, 0, 0, 0, None, None, None, 1, 1, 1, 1, None, None, None, None, 2, 2, 2, 2, 2, 2, 2, 2, None, None, None, None, None, None, None, None, None, 3, 3, 3, 3, None, None, None, None, None, None, None, None, None, 4, 4, 4, 4, 4, 4, 4, 4, 4, None, None, None, None, None, None, None, 5, 5, None, None, None, None, 6, 6, 6, 6, 6, 6, 6, None, None, None, None, None, None, None, None, None, 7, None, None, None, None, None, None, None, None, None, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, None, 10, 10, 10, None, None, 11, 11, None, None, None, None, None, 12, 12, 12, 12, None, None, None, None, None, None, None, 13, 13, 13, None, None, None, None, 14, 14, 14, None, None, 15, 15, None, None, None, None, None, None, None, None, 16, 16, 16, None, None, None, None, None, None, 17, 17, 17, None, None, None, None, 18, 18, 18, 18, 18, 18, None, None, 19, 19, 19, 19, None, None, 20, 20, 21, 21, 21, 21, 21, 21, 21, None, None, None, 22, 22, 22, 22, 22, 22, 22, None, None, None, None, None, None,

### PART 2
(didn't work for final input)

In [ ]:
from collections import defaultdict

f = open('inputs/9.txt')
raw_file = f.read().strip()

id = 0
processed_file = []
isFile = True

for i in raw_file:
    i = int(i)
    if isFile:
        processed_file += [id] * i
        id += 1
        isFile = False
    else: 
        processed_file += [None] * i
        isFile = True

print(processed_file)

# create two datastructures to keep track of free and used blocks
# for free_blocks, key is the amount of free space and value is a list of tuples that contain the start and end position 
# for a free space of that size

# for used_blocks, key is the id of the file and value is the start and end location of that file block
free_blocks = defaultdict(list)
used_blocks = defaultdict(tuple)

# utility function to swap blocks in the processed file
def swap_block(source, target):
    s1, s2 = source
    t1, t2 = target
    processed_file[s1:s2] = processed_file[t1:t2]
    processed_file[t1:t2] = [None] * (t2-t1)

# populate the free_blocks and used_blocks 
i = 0
while i < len(processed_file):
    if processed_file[i] == None:
        start = i
        while processed_file[i] == None:
            i += 1
        end = i
        free_blocks[end-start].append((start, end))
    else: 
        key = processed_file[i]
        start = i
        while i < len(processed_file) and processed_file[i] == key:
            i += 1
        end = i
        used_blocks[key] = (start, end)

# utility function to sort the free_blocks since we always need to pick the first free block
def sort_free_blocks():
    for f in free_blocks:
        free_blocks[f] = sorted(free_blocks[f], key = lambda x: x[0])


sort_free_blocks()

# utility function that takes in a size and returns the first free block available
def get_free_block(size):
    # we loop through all the keys in free_blocks (which are the sizes of the free blocks)
    for f in free_blocks.keys():
        # if we find a size that is <= the size we need AND the list of that size in free_blocks is not empty
        if size <= int(f) and len(free_blocks[f]) > 0:
            # we remove that block from free_blocks
            free_block = free_blocks[f].pop(0)
            free_size = free_block[1] - free_block[0]
            if size != int(f):
                # if size is not exactly equal to the free_block size i.e. required space is less than available space
                # we need to calulate the required block and add the rest of the free block back to the free_blocks
                required_block = (free_block[0], free_block[0] + size)
                free_blocks[free_size - size].append((free_block[0] + size, free_block[1]))

                # after insertion of remaining free block, we need to sort again to make sure we pick the first free block later on
                sort_free_blocks()
                return required_block
            else:
                return free_block
    # if the required size is more than any free space then we do nothing
    return None

# we sort the used_blocks in descending order since we need to move files right to left 
# we could also navigate in reverse to achieve the same thing
keys = sorted(used_blocks.keys(), reverse=True)

for i in keys:
    size = used_blocks[i][1] - used_blocks[i][0]
    source = get_free_block(size)
    target = used_blocks[i]

    if source:
        swap_block(source, target)
    
checksum = 0
for index, s in enumerate(processed_file):
    if s != None:
        checksum += index * s

print(checksum)


[0, 0, 0, 0, 0, 0, 0, 0, None, None, None, 1, 1, 1, 1, None, None, None, None, 2, 2, 2, 2, 2, 2, 2, 2, None, None, None, None, None, None, None, None, None, 3, 3, 3, 3, None, None, None, None, None, None, None, None, None, 4, 4, 4, 4, 4, 4, 4, 4, 4, None, None, None, None, None, None, None, 5, 5, None, None, None, None, 6, 6, 6, 6, 6, 6, 6, None, None, None, None, None, None, None, None, None, 7, None, None, None, None, None, None, None, None, None, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, None, 10, 10, 10, None, None, 11, 11, None, None, None, None, None, 12, 12, 12, 12, None, None, None, None, None, None, None, 13, 13, 13, None, None, None, None, 14, 14, 14, None, None, 15, 15, None, None, None, None, None, None, None, None, 16, 16, 16, None, None, None, None, None, None, 17, 17, 17, None, None, None, None, 18, 18, 18, 18, 18, 18, None, None, 19, 19, 19, 19, None, None, 20, 20, 21, 21, 21, 21, 21, 21, 21, None, None, None, 22, 22, 22, 22, 22, 22, 22, None, None, None, None, None, None,

Code from https://github.com/womogenes/AoC-2024-Solutions/blob/main/day_09/p2_day_09.py that works

In [6]:
from tqdm import tqdm

with open("inputs/9.txt") as fin:
    line = fin.read().strip()

# Allocate a lot of space
size = [0] * len(line)
loc = [0] * len(line)

def make_filesystem(diskmap):
    global loc, size

    blocks = []

    is_file = True
    id = 0
    for x in diskmap:
        x = int(x)
        if is_file:
            loc[id] = len(blocks)
            size[id] = x
            blocks += [id] * x
            id += 1
            is_file = False
        else:
            blocks += [None] * x
            is_file = True
    
    return blocks

filesystem = make_filesystem(line)

def move(arr):
    # Current file to move
    big = 0
    while size[big] > 0:
        big += 1
    big -= 1

    for to_move in tqdm(range(big, -1, -1)):
        # Find first free space that works
        free_space = 0
        first_free = 0
        while first_free < loc[to_move] and free_space < size[to_move]:
            first_free = first_free + free_space
            free_space = 0
            while arr[first_free] != None:
                first_free += 1
            while first_free + free_space < len(arr) and arr[first_free + free_space] == None:
                free_space += 1
        
        if first_free >= loc[to_move]:
            continue
        
        # Move file by swapping block values
        for idx in range(first_free, first_free + size[to_move]):
            arr[idx] = to_move
        for idx in range(loc[to_move], loc[to_move] + size[to_move]):
            arr[idx] = None
        
    return arr

def checksum(arr):
    ans = 0
    for i, x in enumerate(arr):
        if x != None:
            ans += i * x
    return ans

moved = move(filesystem)

print(checksum(moved))

100%|██████████| 10000/10000 [00:23<00:00, 427.52it/s]

6335972980679
